### **Generación de Texto con una RNN** ###

Vamos a utilizar un RNN para [generar texto](https://www.tensorflow.org/text/tutorials/text_generation). Le mostraremos a la red una muestra de lo que queremos y aprenderá cómo escribir una versión por si misma. Para hacerlo, vamos a utilizar un modelo de predicción de caracteres que tomará como entrada una secuencia de longitud variable y predice el siguiente caráter. Si lo utilizamos de manera recurrente, conseguiremos que se cree un texto.

**Modulos:**

In [ ]:
import os
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing import sequence

**Dataset:**

Vamos a utilizar como Dataset de entrada para entrenar nuestro modelo parte de la obra Romeo y Julieta de Shakespeare.


In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

Si quisiéramos, podríamos utilizar un texto propio.

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

**Leer contenido del documento de entrada:**

In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto: {} caracteres'.format(len(text)))
print(text[:250])

**Preprocesar la entrada de datos:**

La entrada de datos está en formato texto. Sin embargo, nuestro modelo necitará una entrada de datos en formato numérico. Por tanto, debemos codificar nuestro texto antes de pasarlo como entrada al modelo. Vamos a codificar cada carácter como un entero.

In [ ]:
vocab = sorted(set(text))                     # Ordenamos cada carácter único existente en el texto e entrada
idx2char = np.array(vocab)                    # Convertimos la lista en un array para poder acceder a cada carácter por su índice
char2idx = {u:i for i, u in enumerate(vocab)} # Nos permitirá obtener el índice de cada carácter

# La siguiente función devuelve un array NumPy creado a partir del texto de entrada
def text_to_int(text):
  return np.array([char2idx[c] for c in text]) # Expresión de list comprehension que ejecuta un bucle For...
  # int_array = []
  # for c in text:
  #   int_array.append(char2idx[c])
  # return np.array(int_array)

text_as_int = text_to_int(text)

# La siguiente función devuelve un Texto creado a partir de un Array NumPy (permite revertir la codificación)
def int_to_text(ints):
  try:
    ints = int.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

In [ ]:
print(vocab)
print(list(enumerate(vocab)))
print(char2idx)
print('Texto:', text[:20])
print('Codificado:', text_as_int[:20])
print(int_to_text(text_to_int(text[:20])))

**Creación de ejemplos:**

El objetivo es pasarle al modelo una secuencia y que nos devuelva el siguiente carácter. Por tanto, necesitamos partir el texto en muchos secuencias cortas para entrenar nuestro modelo.

Los ejemplos de entrenamiento que vamos a crear toman secuencias de *seq_length* como entrada y devuelve secuencias de *seq_length* como salida (secuencia de entrada movida un carácter a la derecha - entrada: Hell | salida: ello-).

In [ ]:
seq_lenght = 100                                # Longitud de las secuencas para entrenamiento del modelo
examples_per_eposch = len(text)//(seq_lenght+1) # Calculamos los ejemplos que obtenemos del texto inicial (+1 en el denominador para recoger el texto restante)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)     # Cargamos todo el texto de entrada en un Tensor (contiene valores enteros)
sequences = char_dataset.batch(seq_lenght+1, drop_remainder=True) # Cada secuencia será de seq_lenght + 1 para contener tanto la entrada de datos como su etiqueta

Una vez disponemos de las secuencas, debemos partirlas en entradas y salidas (etiquetas).

In [ ]:
def split_imput_target(chunk):   # Ejemplo: hello
  input_text = chunk[:-1]        # hell
  target_text = chunk[1:]        # ello
  return input_text, target_text # hell, ello

In [ ]:
dataset = sequences.map(split_imput_target) # Utilizamos el map para aplicar la fucnión a cada elemento

In [ ]:
for x, y in dataset.take(2):
  print('\n\nEXAMPLE\n')
  print('INPUT')
  print(int_to_text(x))
  print('\nOUTPUT')
  print(int_to_text(y))

Por último, debemos preparar los datos de entrada para el modelo.

In [ ]:
BATCH_SIZE = 64         # Cantidad de ejemplos para training por Batch
VOCAB_SIZE = len(vocab) # Cantidad de caracteres únicos existentes en nuestro vocabulario
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000 # Cantidad de elementos de la secuencia que considera para el Suffle

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

**Construcción del Modelo:**

Vamos a utilizar una capa de Embedding, una capa LSTM y una capa Densa con un nodo por cada carácter existente en nuestro diccionario. La capa densa nos devolverá la distribución de probabilidades sobre toos sus nodos.

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,             # Capa de Embedding
                                batch_input_shape=[batch_size, None]), # El None indica que no sabemos el tamaño de cada elemento dentro del Batch
      tf.keras.layers.LSTM(rnn_units,
                           return_sequences=True,
                           stateful=True,
                           recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

In [ ]:
model.summary()

**Función de Pérdida:**

Es neceario definir la funión de pérdia para evaluar el modelo. Esto lo hacemos porque el modelo devolverá como salida un Tensor de (64, 100, 65) (Elementos en el Batch, Tamaño de cada Elemento, Nodos de salida con la distribución de probabilidades). Antes vamos a analizar la salida que se espera del modelo.

In [ ]:
# Ejemplo de salida del modelo (se lo estamos pidiendo al modelo sin entrenamiento)
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch) # Le pedimos al modelo una predicción
  print(example_batch_predictions.shape, '# (batch_size, sequence_lenght, vocab_size)')

In [ ]:
print(len(example_batch_predictions))  # Array de 64 arrays (elementos dentro del Batch)
pred = example_batch_predictions[0]    # Le hemos indicado al modelo que la capa LSTM tenga 'return_sequences=True', por tanto, nos devolverá una distribución de probabiliades por cada Paso de Tiempo (Ejemplo 'Hello': T1 H, T2 He, ...)
print(len(pred))
print(example_batch_predictions[0][0]) # Distribución de probabilidades del primer paso de Tiempo (65 valores numéricos)

Si queremos determinar el carácter predicho, tenemos que elegir un valor basado en la distribución de probabilidades. Esto lo haremos con la función tf.random.categorical.

Dicha función toma las distribuciones de probabilidad que devuelve el modelo y realiza muestreos categóricos, devolviendo los índices de las muestras seleccionadas. Por lo tanto, sampled_indices contendrá los índices de las palabras seleccionadas aleatoriamente según las distribuciones de probabilidad proporcionadas por pred. Estos índices representarán las palabras predichas en la siguiente secuencia generada por el modelo.

In [ ]:
sampled_indices1 = tf.random.categorical(pred, num_samples=1) # Devolvemos una sola muestra para cada distribución de probabilidades (100), que será el carácter predicho

sampled_indices = np.reshape(sampled_indices1, (1, -1))[0]   # Pasamos las predicciones resultantes de cada Paso de Tiempo a un Tensor de Shape (100,0) para pasarlas a la función int_to_char()
predicted_chars = int_to_text(sampled_indices)               # Predicción del siguiente carácter en cada Paso de Tiempo

In [ ]:
print(sampled_indices1.shape) # Vector de 100 elementos con 1 elemento cada uno
print(sampled_indices.shape)  # Vector de 100 elementos
print(sampled_indices1)
print(sampled_indices)
print(predicted_chars)

Vamos a crear una función de Pérdida que nos permita comparar esa salida con la salida esperada y nos devuelva una representación numérica.

In [15]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

**Compilando el Modelo:**

En este momento, podemos pensar en nuestro problema como un problema de clasificación, donde el modelo predice la probabilidad de que cada uno de los posibles carácteres del vocabulario sea el próximo.

In [16]:
model.compile(optimizer='adam', loss=loss)

**Creación de Checkpoints:**

Los Checkpoints son puntos de control en el entrenamiento del modelo en los que se guarda una copia del modelo y su estado en ese momento particular.

Vamos a crear y configurar nuestros Checkpoints. Esto nos permitirá cargar nuestro modelo desde un Checkpoint y continuar con el proceso de entrenamiento a partir de ese punto.

In [17]:
checkpoint_dir = './Training_Checkpoints'                        # Creamos el directorio en el que guardaremos los Checkpoings
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}') # Nombre de los ficheros de Checkpoint

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_wights_only=True
)

**Entrenamiento del Modelo**

In [ ]:
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40
172/172 [==============================] - ETA: 0s - loss: 2.5873 

INFO:tensorflow:Assets written to: ./Training_Checkpoints\ckpt_1\assets


INFO:tensorflow:Assets written to: ./Training_Checkpoints\ckpt_1\assets


172/172 [==============================] - 1871s 11s/step - loss: 2.5873
Epoch 2/40
 52/172 [========>.....................] - ETA: 21:01 - loss: 2.0051